In [ ]:
from datasets import load_dataset
from PIL import Image
import io

# Load your dataset
dataset = load_dataset('cairocode/MSP_POD_SYL')  # Replace with your actual dataset name

In [1]:
from PIL import features

print("JPEG support:", features.check("jpeg"))
print("PNG support:", features.check("png"))
print("WebP support:", features.check("webp"))


JPEG support: False
PNG support: False
WebP support: True


/tmp/ipykernel_102547/2526314912.py:3: UserWarning: Unknown feature 'jpeg'.
  print("JPEG support:", features.check("jpeg"))
/tmp/ipykernel_102547/2526314912.py:4: UserWarning: Unknown feature 'png'.
  print("PNG support:", features.check("png"))


In [2]:
remaining_data = dataset.filter(lambda x: x['speaker'] != 'Unknown')['train']


In [3]:
val_size = int(len(remaining_data)*0.2)

In [4]:
column = 'speaker'

In [5]:
from collections import defaultdict

# Group the data by class
class_groups = defaultdict(list)
for idx, item in enumerate(remaining_data):
    class_groups[item[column]].append(idx)

# Ensure at least one sample from each class in validation set
validation_indices = []
train_indices = []

for class_indices in class_groups.values():
    if len(class_indices) > 1:
        validation_indices.append(class_indices.pop(0))
    train_indices.extend(class_indices)

# Calculate remaining validation size
remaining_val_size = val_size - len(validation_indices)

# Randomly sample the rest of the validation set
import random
random.seed(42)
additional_val_indices = random.sample(train_indices, remaining_val_size)

validation_indices.extend(additional_val_indices)
train_indices = [idx for idx in train_indices if idx not in additional_val_indices]

# Create the datasets
validation_dataset = remaining_data.select(validation_indices)
train_dataset = remaining_data.select(train_indices)

In [ ]:
train_dataset

In [7]:
column = 'speaker'

In [8]:
S = train_dataset[column]
def adjust_speaker_ids(example, min_val):
    example[column] = int(example[column]) - min_val
    return example
    
unique_labels = list(set(S))

min_val = min(int(label) for label in unique_labels)  # If the labels are strings representing integers
Cval = len(unique_labels)

train_dataset = train_dataset.map(lambda example: adjust_speaker_ids(example, min_val))


validation_dataset = validation_dataset.map(lambda example: adjust_speaker_ids(example, min_val))
S = train_dataset[column]
unique_labels = list(set(S))


speaktonum = {label: str(index) for index, label in enumerate(unique_labels)}
# Create a dictionary that maps numbers to labels
numtospeak = {str(index): label for index, label in enumerate(unique_labels)}

In [9]:
test_dataset = dataset.filter(lambda x: x['speaker'] == 'Unknown')['train']


In [10]:
import numpy as np
from datasets import Value

def convert_speaker(value):
    if value == 'Unknown':
        return None
    try:
        return int(value)
    except ValueError:
        return None

# Convert the 'speaker' column
test_dataset = test_dataset.map(
    lambda x: {'speaker': convert_speaker(x['speaker'])},
    remove_columns=['speaker']
)

# Set the column type to Int64
test_dataset = test_dataset.cast_column('speaker', Value('int64'))

In [11]:
from datasets import DatasetDict
train_d1 = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

In [ ]:

train_d1.push_to_hub("cairocode/MSP_Pod_SYL4")

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict


train_dataset = load_dataset('cairocode/MSP_POD_OLD_5')

In [ ]:
train_dataset

In [2]:
column = 'EmoClass'

In [3]:
train_data = train_dataset['train']

In [ ]:
corrupted_files = []
try:
    for idx, sample in enumerate(train_data):
        try:
            label = sample[column]
            img = sample['image']
        except OSError as e:
            print(f"Error in sample {idx}: {e}")
            corrupted_files.append(idx)
except OSError as e:
    print(f"Error in sample {idx}: {e}")
    corrupted_files.append(idx+1)

print(f"Corrupted files indices: {corrupted_files}")

In [ ]:
import PIL
import tqdm

for idx in tqdm.trange(len(train_data)):
    try:
        _ = train_data[idx]["image"].load()
    except PIL.UnidentifiedImageError as e:
        print(f"Corrupted image at index={idx}: {e}")


In [ ]:

# corrupted_files[0] = corrupted_files[0] - 1
corrupted_files

In [ ]:
if corrupted_files:
    train_dataset_new = train_data.select([i for i in range(len(train_data)) if i not in corrupted_files])
    print(f"Removed {len(corrupted_files)} corrupted files from the dataset.")
else:
    print("No corrupted files found.")

train_data = train_dataset_new

In [ ]:
len(train_dataset_new)

In [ ]:
from datasets import DatasetDict
train_d1 = DatasetDict({
    'train': train_dataset_new,
    # 'validation': train_dataset['validation'],
    # 'test': train_dataset['test']
})

train_d1.push_to_hub("MSP_POD_OLD_5")

In [ ]:

# Function to try opening an image
from datasets import load_dataset
from PIL import Image
import io
import numpy as np
def try_open_image(image_data):
    try:
        if isinstance(image_data, dict) and 'bytes' in image_data:
            # If it's a dict with 'bytes' key, use that
            image_data = image_data['bytes']
        
        if isinstance(image_data, bytes):
            # If it's bytes, try to convert to numpy array and then to PIL Image
            nparr = np.frombuffer(image_data, np.uint8)
            img = Image.fromarray(nparr)
        elif isinstance(image_data, np.ndarray):
            # If it's already a numpy array, convert to PIL Image
            img = Image.fromarray(image_data)
        elif isinstance(image_data, Image.Image):
            # If it's already a PIL Image, use it directly
            img = image_data
        else:
            raise ValueError(f"Unexpected image data type: {type(image_data)}")
        
        # Try to convert to RGB to ensure it's a valid image
        img.convert('RGB')
        return True
    except Exception as e:
        return str(e)

# List to store indices of valid examples
valid_indices = []

# Loop through the dataset
for idx, example in enumerate(dataset):
    # Assuming the image is stored in a column named 'image'
    # Adjust this if your image column has a different name
    result = try_open_image(example['image'])
    
    if result is True:
        valid_indices.append(idx)
    else:
        print(f"Issue with image at index {idx}: {result}")

# Create a new dataset with only the valid examples
cleaned_dataset = dataset.select(valid_indices)

print(f"Original dataset size: {len(dataset)}")
print(f"Cleaned dataset size: {len(cleaned_dataset)}")


In [ ]:
from dataset import load_dataset
dataset -= load_dataset('\')